In [50]:
import pandas as pd
import numpy as np
import os
from functools import partial

In [18]:
root = os.path.dirname(os.getcwd())
data_dir = os.path.join(root, 'data')
snapshot_path = os.path.join(data_dir, 'snapshot_data.csv')

In [47]:
pd.read_csv(snapshot_path)

,City,POP,GDP,PRIM,SEC,TERT,>60yr%,BED,DOC,NRS,...,Prim%,Sec%,Tert%,ACTV,TVLR,TVLR‰,Cases,MORB%,log(Cases),log(MORB%)
0,Shenyang,832.20,6470.30,284.00,2178.60,4007.60,0.232683,8.643235,3.815068,4.589161,...,0.043893,0.336708,0.619384,5.603786,0.376150,0.000452,36,0.000004,3.583519,-12.350894
1,Dalian,710.00,7001.70,458.50,2799.80,3743.30,0.228510,6.648000,3.139000,3.698000,...,0.065484,0.399874,0.534627,5.601579,0.095776,0.000135,27,0.000004,3.295837,-12.479768
2,Chaoyang,304.46,843.10,194.60,248.00,400.50,0.246732,6.550614,2.672929,2.511660,...,0.230815,0.294153,0.475033,5.827807,0.000000,0.000000,6,0.000002,1.791759,-13.137121
3,Jinzhou,294.96,1073.00,198.30,279.20,595.60,0.280787,6.371372,3.266884,4.131069,...,0.184809,0.260205,0.555079,5.451550,0.000000,0.000000,12,0.000004,2.484907,-12.412273
4,Huludao,253.79,807.10,133.10,302.20,371.80,0.259177,6.585760,2.088341,3.096827,...,0.164911,0.374427,0.460662,5.713643,0.000000,0.000000,12,0.000005,2.484907,-12.261941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,Jiangmen,463.03,3146.64,254.23,1352.54,1539.87,0.185380,5.236810,2.361402,3.243418,...,0.080794,0.429836,0.489370,4.456414,0.000000,0.000000,24,0.000005,3.178054,-12.170078
162,Zhaoqing,418.71,2248.80,386.02,925.45,937.33,0.170045,4.412839,2.023119,2.690645,...,0.171656,0.411531,0.416813,4.341986,0.000000,0.000000,20,0.000005,2.995732,-12.251787
163,Huizhou,488.00,4177.41,205.50,2169.12,1802.79,0.098566,4.477869,2.867213,3.272336,...,0.049193,0.519250,0.431557,3.509514,0.278907,0.000572,62,0.000013,4.127134,-11.273521
164,Dongguan,846.45,9482.50,28.48,5361.50,4092.52,0.049152,3.898635,2.325493,3.032816,...,0.003003,0.565410,0.431587,2.984629,0.675752,0.000798,100,0.000012,4.605170,-11.346221


In [143]:
# generate summary statistics
column_name_map = {
                   'POP':'Population (in thousands)',
                   'Area':'City area (in km^2)',
                   'POPDENS':'Population Density (people per km^2)',
                   'GDP':'GDP (Billions USD)',
                   'PRIM':'Primary sector (Billions USD)',
                   'SEC':'Secondary sector (Billions USD)',
                   'TERT':'Tertiary sector (Billions USD)',
                   'Prim%':'Primary sector % of GDP',
                   'Sec%':'Secondary sector % of GDP',
                   'Tert%':'Tertiary sector % of GDP',
                   'GDPpc':'GDP per capita (Billions USD per km^2)',
                   '>60yr%':'Elderly population %',
                   'BED':'Hospital Beds (per thousand people)',
                   'DOC':'Registered doctors (per thousand)',
                   'NRS':'Registered nurses (per thousand)',
                   'TVLR':'Wuhan travellers (thousands)',
                   'TVLR‰':'Wuhan travellers (per thousand pop.)',
                   'ACTV':'Average degree of activeness (0-8)'
                  }
'', 'std', 'amin', 'percentile', 'median', 'percentile', 'amax'
snapshots = pd.read_csv(snapshot_path)
snapshots['POP'] *= 10
snapshots['POPDENS'] *= 10000
snapshots['Prim%'] *= 100
snapshots['Sec%'] *= 100
snapshots['Tert%'] *= 100
snapshots['>60yr%'] *= 100
snapshots['TVLR'] *= 10
snapshots['TVLR‰'] *= 1000
snapshots = snapshots.drop(
    columns=[x for x in snapshots.columns if x not in column_name_map]).rename(
    column_name_map, axis=1)
first_quartile = partial(np.percentile, q=25, axis=0)
third_quartile = partial(np.percentile, q=75, axis=0)
summary_stats  = snapshots.apply([np.mean, np.std, np.min, first_quartile,
                 np.median, third_quartile, np.max],
                 axis=0, result_type='broadcast').T.round(2)
summary_stats.columns = ['Mean',
                         'SD',
                         'Min',
                         '25th %ile',
                         'Median',
                         '75th %ile',
                         'Max']
summary_stats['IQR'] = summary_stats['75th %ile'] - summary_stats['25th %ile']
summary_stats

,Mean,SD,Min,25th %ile,Median,75th %ile,Max,IQR
Population (in thousands),5624.67,4029.79,720.96,3176.92,4666.55,7181.67,31243.20,4004.75
GDP (Billions USD),4561.69,5633.80,354.72,1599.24,2759.78,4979.38,38155.32,3380.14
Primary sector (Billions USD),244.45,173.23,11.80,133.27,216.35,321.06,1511.42,187.79
Secondary sector (Billions USD),1782.75,1861.46,130.54,704.88,1120.20,2072.60,10495.84,1367.72
Tertiary sector (Billions USD),2534.45,3942.48,197.22,796.67,1319.62,2465.34,29542.50,1668.67
Elderly population %,19.51,4.51,4.92,17.13,19.69,22.48,32.20,5.35
Hospital Beds (per thousand people),6.22,1.22,3.82,5.43,6.10,6.90,9.67,1.47
Registered doctors (per thousand),2.81,0.76,1.32,2.29,2.73,3.14,5.76,0.85
Registered nurses (per thousand),3.19,1.01,1.27,2.51,3.03,3.60,6.72,1.09
City area (in km^2),11733.64,9080.77,1459.00,6339.50,10238.00,14288.50,82402.00,7949.00


### Notes
- The original data doesn't measure in same units as summary table 
    - Needed to convert to in thousands and proportions to percentages 

- The GDP measurements are off (probably due to different data being used)
    - GDP
    - Primary sector
    - Secondary sector 
    - Tertiary sector 
    - Primary sector %
    - Secondary sector %
    - Tertiary sector %
    - GDP per capita
    
- Wuhan traveller measurements are off (probably due to different data being used)
    - Wuhan travellers
    - Wuhan traveller %
    
- Need to append ```\usepackage{booktabs}``` before using LaTeX representation of table

In [144]:
print(summary_stats.to_latex(index = True, multirow = True, bold_rows=True))

\begin{tabular}{lrrrrrrrr}
\toprule
{} &      Mean &       SD &      Min &  25th \%ile &    Median &  75th \%ile &       Max &      IQR \\
\midrule
\textbf{Population (in thousands)             } &   5624.67 &  4029.79 &   720.96 &    3176.92 &   4666.55 &    7181.67 &  31243.20 &  4004.75 \\
\textbf{GDP (Billions USD)                    } &   4561.69 &  5633.80 &   354.72 &    1599.24 &   2759.78 &    4979.38 &  38155.32 &  3380.14 \\
\textbf{Primary sector (Billions USD)         } &    244.45 &   173.23 &    11.80 &     133.27 &    216.35 &     321.06 &   1511.42 &   187.79 \\
\textbf{Secondary sector (Billions USD)       } &   1782.75 &  1861.46 &   130.54 &     704.88 &   1120.20 &    2072.60 &  10495.84 &  1367.72 \\
\textbf{Tertiary sector (Billions USD)        } &   2534.45 &  3942.48 &   197.22 &     796.67 &   1319.62 &    2465.34 &  29542.50 &  1668.67 \\
\textbf{Elderly population \%                  } &     19.51 &     4.51 &     4.92 &      17.13 &     19.69 &      22.48 &